In [110]:
import subprocess
import time
import os 
import sys 
import io
import functions
import importlib
import inspect

import warnings 
warnings.filterwarnings('ignore')

import pandas as pd 
import numpy as np


In [119]:
importlib.reload(functions)
from functions import *

# Get all functions in the module
functions_list = [f[0] for f in inspect.getmembers(functions, inspect.isfunction)]
print(functions_list)

['acf', 'add_constant', 'add_values', 'adfuller', 'ar_select_order', 'confusion_matrix', 'create_TS_data', 'create_VAR_irf_model', 'create_VAR_model', 'get_granger', 'get_mip', 'get_public_opinion_data', 'granger_table', 'make_attention_plot', 'make_mip_plot', 'make_subplots', 'mip', 'mip_data', 'pacf', 'plot_irf', 'plot_irf_v2', 'print_MIP_descriptive_stats', 'print_confusion_matrix', 'return_mip_data', 'select_lag_order', 'variance_inflation_factor']


# Confusion Matrix (Appendix)

In [124]:
print_confusion_matrix('confusion_matrix_data.csv')

In [123]:
# return full combined dataset 
return_mip_data()

,date,issue,priority,gender,country
0,2009-01-20,education,0.04,male,us
1,2009-06-21,education,0.02,male,us
2,2009-06-28,education,0.06,male,us
3,2009-07-26,education,0.08,male,us
4,2009-08-02,education,0.03,male,us
...,...,...,...,...,...
19563,2023-12-11,defense,0.06,female,uk
19564,2023-12-18,defense,0.07,female,uk
19565,2023-12-24,defense,0.07,female,uk
19566,2024-01-01,defense,0.07,female,uk


In [115]:
print_MIP_descriptive_stats()

\begin{tabular}{lllllllllll}
\toprule
Country & Issue & Gender & count & mean & std & min & 25% & 50% & 75% & max \\
\midrule
UK & Crime & female & 182.00 & 0.19 & 0.07 & 0.07 & 0.14 & 0.20 & 0.24 & 0.39 \\
UK & Crime & male & 182.00 & 0.19 & 0.06 & 0.09 & 0.14 & 0.19 & 0.23 & 0.38 \\
UK & Defense & female & 182.00 & 0.09 & 0.04 & 0.03 & 0.06 & 0.08 & 0.12 & 0.29 \\
UK & Defense & male & 182.00 & 0.11 & 0.04 & 0.05 & 0.08 & 0.10 & 0.13 & 0.25 \\
UK & Economy & female & 182.00 & 0.36 & 0.15 & 0.18 & 0.22 & 0.32 & 0.53 & 0.61 \\
UK & Economy & male & 182.00 & 0.42 & 0.12 & 0.27 & 0.31 & 0.40 & 0.54 & 0.65 \\
UK & Education & female & 182.00 & 0.16 & 0.04 & 0.10 & 0.13 & 0.15 & 0.17 & 0.32 \\
UK & Education & male & 182.00 & 0.12 & 0.03 & 0.06 & 0.10 & 0.11 & 0.13 & 0.24 \\
UK & Environment & female & 182.00 & 0.24 & 0.08 & 0.10 & 0.18 & 0.25 & 0.30 & 0.42 \\
UK & Environment & male & 182.00 & 0.23 & 0.07 & 0.08 & 0.18 & 0.24 & 0.28 & 0.38 \\
UK & Health & female & 182.00 & 0.53 & 0.11 & 

# Public Opinion Data

In [104]:
# plot US 

make_mip_plot(mip = return_mip_data(), 
              col_wrap=2,
              country = 'us')
              

In [105]:
# plot UK 

make_mip_plot(mip = return_mip_data(), 
              col_wrap=3,
              country = 'uk')

# Legislators' attention

In [112]:
make_attention_plot('uk', .12)

In [113]:
make_attention_plot('us', .12)

# Vector Autorregressions 

In [41]:
# read in data 

path = '../data/attention_time_series_v2.csv'

us = create_TS_data(path, 'us')
uk = create_TS_data(path, 'uk')

# Fit VAR Models with IRFs 

In [42]:
# create VAR models and IRFs

us_model, us_irf = create_VAR_model(data = us,
                            lags = 10,
                            irf_horizon = 4)

uk_model, uk_irf = create_VAR_model(data = uk,
                            lags=10,
                            irf_horizon = 4)

# Plot IRFs using VAR models

In [57]:
# plot US IRF 

plot_irf(us_model, 
         us_irf, 
         estimate = 'cumulative_effects')

(0.0, 7.18)
(0.0, 6.892)
(0.0, 7.049)
(0.001, 2.94)
(0.0, 6.086)
(0.0, 4.542)
(0.001, 2.981)
(0.002, 2.827)
(0.0, 6.329)
(0.015, 2.202)
(0.0, 4.01)
(0.0, 9.286)


In [61]:
# plot US IRF 

plot_irf(uk_model, 
         uk_irf, 
         estimate = 'cumulative_effects')

(0.001, 2.996)
(0.0, 5.471)
(0.0, 7.996)
(0.0, 3.328)
(0.0, 6.408)
(0.0, 5.381)
(0.0, 4.414)
(0.297, 1.189)
(0.0, 4.137)
(0.0, 4.152)
(0.0, 3.304)
(0.0, 10.76)


# Granger Causality Tables in Appendix 

In [59]:
# print granger table for appendix for US model 

granger_table(us_model)

\begin{tabular}{lrrrl}
\toprule
Coefficient & Test statistic & p-value & Critical value & df \\
\midrule
Men Reps' Attention -> Men Reps' Attention & 17.160000 & 0.000000 & 1.832000 & (10, 6264) \\
Men Reps' Attention -> Women Reps' Attention & 2.940000 & 0.001000 & 1.832000 & (10, 6264) \\
Men Reps' Attention -> Women's Salience & 2.981000 & 0.001000 & 1.832000 & (10, 6264) \\
Men Reps' Attention -> Men's Salience & 2.202000 & 0.015000 & 1.832000 & (10, 6264) \\
Women Reps' Attention -> Men Reps' Attention & 7.180000 & 0.000000 & 1.832000 & (10, 6264) \\
Women Reps' Attention -> Women Reps' Attention & 7.639000 & 0.000000 & 1.832000 & (10, 6264) \\
Women Reps' Attention -> Women's Salience & 2.827000 & 0.002000 & 1.832000 & (10, 6264) \\
Women Reps' Attention -> Men's Salience & 4.010000 & 0.000000 & 1.832000 & (10, 6264) \\
Women's Salience -> Men Reps' Attention & 6.892000 & 0.000000 & 1.832000 & (10, 6264) \\
Women's Salience -> Women Reps' Attention & 6.086000 & 0.000000 & 1.83200

In [60]:
# print granger table for appendix for US model 

granger_table(uk_model)

\begin{tabular}{lrrrl}
\toprule
Coefficient & Test statistic & p-value & Critical value & df \\
\midrule
Men Reps' Attention -> Men Reps' Attention & 4.360000 & 0.000000 & 1.881000 & (9, 6020) \\
Men Reps' Attention -> Women Reps' Attention & 3.328000 & 0.000000 & 1.881000 & (9, 6020) \\
Men Reps' Attention -> Women's Salience & 4.414000 & 0.000000 & 1.881000 & (9, 6020) \\
Men Reps' Attention -> Men's Salience & 4.152000 & 0.000000 & 1.881000 & (9, 6020) \\
Women Reps' Attention -> Men Reps' Attention & 2.996000 & 0.001000 & 1.881000 & (9, 6020) \\
Women Reps' Attention -> Women Reps' Attention & 2.459000 & 0.009000 & 1.881000 & (9, 6020) \\
Women Reps' Attention -> Women's Salience & 1.189000 & 0.297000 & 1.881000 & (9, 6020) \\
Women Reps' Attention -> Men's Salience & 3.304000 & 0.000000 & 1.881000 & (9, 6020) \\
Women's Salience -> Men Reps' Attention & 5.471000 & 0.000000 & 1.881000 & (9, 6020) \\
Women's Salience -> Women Reps' Attention & 6.408000 & 0.000000 & 1.881000 & (9, 60